In [2]:
import logging

from google.cloud import storage

client = storage.Client.from_service_account_json('service/mktg-test-cloud-storage.json')

bucket = client.get_bucket('mktg-data-bucket-1')



In [26]:
from pymongo import MongoClient

# establish mongoDB connection
mongo_con='mongodb://adminProd:{}@planningtool-shard-00-00-k7wis.mongodb.net:27017,planningtool-shard-00-01-k7wis.mongodb.net:27017,planningtool-shard-00-02-k7wis.mongodb.net:27017/test?ssl=true&replicaSet=PlanningTool-shard-0&authSource=admin&w=majority'
mongo_password='Iu25kjB7Hyke0dS1'
mongo_env='production'

mongodb = MongoClient(mongo_con.format(mongo_password))[mongo_env]

channels = dict()
for c in list(mongodb.channels.find({'active':True}, {'prettyName':1})):
    channels[c['_id']] = c['prettyName']
    
regions = dict()
for r in list(mongodb.divisions.find({'active':True}, {'prettyName':1})):
    regions[r['_id']] = r['prettyName']
    
markets = dict()
states = dict()
market_regions = dict()
for m in list(mongodb.markets.find({'active':True}, {'prettyName':1, 'state':1, 'divisionId':1})):
    markets[m['_id']] = m['prettyName']
    states[m['_id']] = m['state']
    market_regions[m['_id']] = regions[m['divisionId']]

departments = dict()
for d in list(mongodb.departments.find({'active':True}, {'prettyName':1})):
    departments[d['_id']] = d['prettyName']

brands = dict()
for b in list(mongodb.brands.find({'active':True}, {'prettyName':1})):
    brands[b['_id']] = b['prettyName']

fiscalYears = dict()
for f in list(mongodb.fiscalyearplans.find({'active':True}, {'fiscalYear':1})):
    fiscalYears[f['_id']] = f['fiscalYear']
    
programs = []
for x in mongodb.programs.aggregate([
            {'$unwind': '$monthItems'},
            {'$match': {
                'monthItems.targetCount': {'$gt': 0},
                'fiscalYearPlanId':{'$in':list(fiscalYears.keys())},
                'active': True,
                }
            },
            {'$project': {
                '_id': 0,
                'fiscalYearPlanId': 1,
                'marketId': 1,
                'channelId': 1,
                'departmentId':1,
                'prettyName': 1,
                'month': '$monthItems.monthNumber',
                'monthName': '$monthItems.prettyName',
                'count': '$monthItems.targetCount',
                'cost': '$monthItems.eventCost',
                'kpi':'$monthItems.kpi'
            }
            }
        ]):
    program = dict()
    program['fiscal'] = fiscalYears[x['fiscalYearPlanId']]
    program['market'] = markets[x['marketId']]
    program['state'] = states[x['marketId']]
    program['region'] = market_regions[x['marketId']]
    program['department'] = departments[x['departmentId']]
    program['channel'] = channels[x['channelId']]
    program['month'] = x['month']
    if x['month'] >= 10:
        program['quarter'] = 2
    elif x['month']>=7:
        program['quarter'] = 1
    elif x['month']>=4:
        program['quarter'] = 4
    elif x['month']>=1:
        program['quarter'] = 3
        
    program['event'] = x['count']
    program['cost'] = x['cost']
    program['kpi'] = x['kpi']
    
    programs.append(program)
    
df = pd.DataFrame(programs)

df

,fiscal,market,state,region,department,channel,month,quarter,event,cost,kpi
0,FY20,Missouri,MO,Central,National Accounts,Off Premise,3,3,2,0.00,0.0
1,FY20,Missouri,MO,Central,National Accounts,Off Premise,4,4,3,0.00,0.0
2,FY20,Colorado,CO,West,National Accounts,Off Premise,3,3,7,0.00,0.0
3,FY20,Colorado,CO,West,National Accounts,Off Premise,4,4,8,0.00,0.0
4,FY20,Wisconsin,WI,Central,National Accounts,Off Premise,3,3,25,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...
33261,FY21 Q1,New Jersey,NJ,East,Central Sampling,Off Premise,8,1,1,250.90,0.0
33262,FY21 Q1,New Jersey,NJ,East,Central Sampling,Off Premise,9,1,1,250.90,0.0
33263,FY21 Q1,Nevada,NV,East,Central Sampling,Off Premise,9,1,8,361.47,0.0
33264,FY21 Q1,Colorado,CO,West,Central Sampling,Off Premise,9,1,3,210.73,0.0


In [27]:
import datetime as dt
import pandas as pd
from io import StringIO # if going with no saving csv file

filename = 'planning-{}.csv'.format(str(dt.date.today()))

f = StringIO()
df.to_csv(f)
f.seek(0)

bucket.blob(filename).upload_from_file(f, content_type='text/csv')